In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

In [2]:
data  = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
Col = list(data.columns)
Col.remove('Personal Loan')
Col.append('Personal Loan')
data = data[Col].drop(columns='ZIP Code')

In [4]:
from sklearn.model_selection import train_test_split
X = data.values[:,:-1]
X = X.reshape(X.shape[0],1,X.shape[1])
Y = data.values[:,-1].reshape(X.shape[0],1)
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)

In [5]:
w_inp = np.random.rand(1,12)
w_hid = np.random.rand(1,4)

In [6]:
def Sigmoid(x):
    s = 1 / (1+np.exp(-1*x))
    return s

In [7]:
def dri_Sigmoid(x):
    s = Sigmoid(x) * (1-Sigmoid(x))
    return s

In [8]:
def model(w1,w2,x_train,y_train,lr=0.03):
    
    # Forward Propagation
    inp_out1 = w1*x_train
    inp_out2 = inp_out1.reshape(inp_out1.shape[0],4,3).sum(axis=2)
    inp_out3 = Sigmoid(inp_out2)
    hid_out1 = w2*inp_out3
    hid_out2 = hid_out1.reshape(hid_out1.shape[0],1,4).sum(axis=2)
    hid_out3 = Sigmoid(hid_out2)
    # Backward Propagation
    Err = hid_out3 - y_train
    dw_hid = -1 * dri_Sigmoid(hid_out1) * inp_out3
    dw_hid = (dw_hid.sum(axis=0)/hid_out3.shape[0]).reshape(1,4)
    w2 = w_hid + (lr * dw_hid)
    w_n = (np.ones((4,3)) * (w2.reshape(4,1)) ).reshape(1,12)
    dw_inp = ((-1 * (w_n) * (dri_Sigmoid(inp_out1)) * x_train ).sum(axis=0) / (hid_out3.shape[0]) ).reshape(1,12)
    w1 = w1 + (lr *dw_inp)
    # w_hid = w_hid - lr* 
    out = {
        "w_inp" : w1,
        "w_hid" : w2,
    }
    return out
# Accuracy Score:  89.28

In [12]:
def Predict(S,x_test,y_test):
    from sklearn.metrics import accuracy_score
    wI = S['w_inp']
    wH = S['w_hid']
    inp_out = (wI*x_test)
    inp_out = inp_out.reshape(inp_out.shape[0],4,3).sum(axis=2)
    inp_out = Sigmoid(inp_out)
    hid_out = wH*inp_out
    hid_out = hid_out.reshape(hid_out.shape[0],1,4).sum(axis=2)
    hid_out = np.floor(Sigmoid(hid_out))
    # print(hid_out)
    return accuracy_score(hid_out,y_test)*100,hid_out

In [13]:
S = model(w_inp,w_hid,x_train,y_train)

In [14]:
Output,s = Predict(S,x_test,y_test)
print("Accutacy is :",Output)

Accutacy is : 90.72


# END